In [ ]:
import os, sys, pathlib, importlib
sys.path.append('../')

# Load the package and modules for training and plotting
import nsbi_common_utils
from nsbi_common_utils import plotting, training
from nsbi_common_utils.training import TrainEvaluate_NN, TrainEvaluatePreselNN


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Nadam
import mplhep as hep
import matplotlib.pyplot as plt
import pickle
import yaml
import random

from utils import preselection_using_score, calculate_preselection_observable

from coffea.analysis_tools import PackedSelection

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

hep.style.use(hep.style.ATLAS)

In [ ]:
with open("config_new.yml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
# path prefix for saving cached data used between modules
path_prefix = config['path_prefix']
path_saved_data = config['path_saved_data']
path_preselection_NN_model = config['path_preselection_NN_model']

# Path for saving interemdiate objects, like NN predictions
saved_data = f'{path_prefix}{path_saved_data}'

# Path for saving/loading preselection NN model
path_to_saved_presel_model = f'{path_prefix}{path_preselection_NN_model}'

In [ ]:
# Get the dictionary of labels to processes
labels_dict = config["labels_dict"]

# Signal processes in the model
signal_processes = config["signal_processes"]

# Background processes in the model
background_processes = config["background_processes"]

print(signal_processes)
print(background_processes)

In [ ]:
# If the preselection NN has already been trained and saved, load from the saved model
USE_SAVED_MODEL_PRESEL = True

# If the preselection NN has already been trained and evaluated, load the numpy array of predictions
USE_SAVED_PRESEL_PREDICTIONS = True

# Input features for training
features = config["features"]

# Subset of the features to standardize before training
features_scaling = config["features_scaling"]

# float factors multiplying the probabilities in the preselection score observable 
pre_factor_preselection_score = config["pre_factor_preselection_score"]


In [ ]:
# Load the nominal dataset saved from the pre-processing notebook

path_to_nominal_dataframe = config["path_to_nominal_dataframe"]
dataset_incl_nominal = pd.read_hdf(path_to_nominal_dataframe, key="dataset", mode='r')

In [ ]:
# Load the MC/data weights and training labels identifying different processes
weights         = dataset_incl_nominal["weights"].to_numpy()
train_labels    = dataset_incl_nominal["train_labels"].to_numpy()

In [ ]:
# Normalizing the training weights - only discriminating shapes
weights_normed  = weights.copy()

for key in labels_dict:

    weights_normed[train_labels==labels_dict[key]] /= weights[train_labels==labels_dict[key]].sum()

dataset_incl_nominal['weights_normed'] = weights_normed


Training the Preselection NN
===

**Choice of reference sample**

The density ratios need to be trained on phase space regions with support for the reference hypothesis $p_{ref}(x) > 0$.

To ensure this, we make a selection that selects events in the phase space regions with $p_{ref}(x) > 0$, or $p_c(x) \gg p_{ref}(x)$, and only perform the NSBI fit in this selected analysis region. **A natural choice for the reference hypothesis is then the signal-rich hypotheses**. This is referred to in the ATLAS publications as the Search-Oriented Mixture Models approach: 

$$p_{ref}(x) = \frac{1}{\sum_S \nu_S} \sum_S \frac{d\sigma_S}{dx} = \frac{1}{\nu_{H \to \tau\tau}} \frac{d\sigma_{H \to \tau\tau}}{dx}$$

where the sum runs over all signal hypothesis in the model and the second equality is due to the sole signal hypothesis in our toy model, $pp \to {t\bar{t}}$. 

In [ ]:
# What are the signal processes in the user-provided model?
print(signal_processes)

In [ ]:
# The reference hypothesis is chosen as the sum of signal hypothesis
ref_processes = config["ref_processes"]
print(ref_processes)

**Selecting out regions with $p_{ref}\sim 0$**

A multi-class classification NN, with softmax output, is trained to output a score:

$$ \text{NN}_\text{presel} = \log \left[\frac{\sum_S P_S (x)}{\sum_B P_B(x)} \right]$$

where $P_c$ are the probability scores outputted from the softmax layer of the trained NN.



In [ ]:
importlib.reload(sys.modules['nsbi_common_utils.training'])
from nsbi_common_utils.training import TrainEvaluatePreselNN

num_classes = len(labels_dict)

preselectionTraining = TrainEvaluatePreselNN(dataset_incl_nominal, 
                                            num_classes, 
                                            features, 
                                            features_scaling)

In [ ]:
if USE_SAVED_PRESEL_PREDICTIONS:

    pred_NN_incl = np.load(f"{saved_data}pred_NN_incl.npy")
    presel_score = calculate_preselection_observable(pred_NN_incl, labels_dict, signal_processes, background_processes, pre_factor_dict = pre_factor_preselection_score)

else:
    if not USE_SAVED_MODEL_PRESEL:
        preselectionTraining.train(test_size=0.2, 
                                   random_state=42, 
                                   path_to_save=path_to_saved_presel_model,
                                  batch_size=4096,
                                  epochs=50, learning_rate=0.1)
    
    else:
        preselectionTraining.get_trained_model(path_to_saved_presel_model)

    # Get predictions (softmax outputs)
    pred_NN_incl = preselectionTraining.predict(dataset_incl_nominal)

    presel_score = calculate_preselection_observable(pred_NN_incl, labels_dict, signal_processes, background_processes, pre_factor_dict = pre_factor_preselection_score)

    np.save(f"{saved_data}presel_score.npy", presel_score)
    np.save(f"{saved_data}pred_NN_incl.npy", pred_NN_incl)

In [ ]:
min_pred = np.amin(presel_score)
max_pred = np.amax(presel_score)

bins = np.linspace(min_pred, max_pred, num=50)

hist_NN_output = {}
hist_NN_output_errs = {}

for key in labels_dict: 
    hist_NN_output[key], _ = np.histogram(presel_score[train_labels==labels_dict[key]], 
                                          weights = weights[train_labels==labels_dict[key]], bins=bins)
    
    hist_NN_output_errs[key], _ = np.histogram(presel_score[train_labels==labels_dict[key]], 
                                          weights = weights[train_labels==labels_dict[key]]**2, bins=bins)


for key in labels_dict:  
    hep.histplot(hist_NN_output[key], bins=bins, 
             alpha=0.6, label=key, 
             density=True, linewidth=2.0, yerr = np.sqrt(hist_NN_output_errs[key]))

plt.xlabel("Preselection Score", size=18)
plt.ylabel("Density", size=18)
plt.legend()
plt.yscale('log')
plt.show()

Making the cut
===

Make a selection cut for regions with $p_{ref} \gg 0$ for performing the NSBI analysis. The remaining events - which by definition are background-dominated - can be used as a **Control Region** for data-driven background estimation, pre-unblinding validations, etc. 

Moreover, the preselections act as a tuning know for the tradeoff in selecting as many signal events as possible to go into the **Signal Region** (increasing sensitivity) and the feasibility of training accurate and precise NNs over a large phase space (need bigger models and more statistics). **The preselections can also weed out phase space regions with low background statistics to avoid poorly modelled regions.** 

Heres a first cut that you can optimize as much as you like to get the desired final results. 

In [ ]:
# Play around with these selections - decrease if estimators are unbiased but need more sensitivity and increase if the model is biased to reduce complexity
preselection_cuts = {'upper': 4.5, 'lower': -1.}
np.save(f"{saved_data}preselection_cuts.npy", preselection_cuts)

for key in labels_dict:  
    hep.histplot(hist_NN_output[key], bins = bins, 
             alpha = 0.6, label = key, 
             density = True, linewidth = 2.0, 
                 yerr = np.sqrt(hist_NN_output_errs[key]))

plt.xlabel("Preselection Score", size=18)

for key in preselection_cuts:
    if preselection_cuts[key] != -999:
        plt.axvline(preselection_cuts[key], ymax=0.9, linestyle='--', label=f'preselection cut {key} = {preselection_cuts[key]}')

plt.ylabel("Density", size=18)
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
for key in labels_dict:  
    hep.histplot(hist_NN_output[key], bins = bins, 
             alpha = 0.6, label = key, 
             density = False, linewidth = 2.0, 
                 yerr = np.sqrt(hist_NN_output_errs[key]))

plt.xlabel("Preselection Score", size=18)

for key in preselection_cuts:
    if preselection_cuts[key] != -999:
        plt.axvline(preselection_cuts[key], ymax=0.9, linestyle='--', label=f'preselection cut {key} = {preselection_cuts[key]}')

plt.ylabel("Density", size=18)
plt.legend()
plt.yscale('log')
plt.show()

Signal and Control Regions
===

The high signal over background phase space towards the right of the preselection cut shown above will be categorized as the **Signal Region** where the NSBI analysis is performed.

The low signal phase space towards the left will be used as a **Control Region**, with typical uses such as background estimation, pre-unblinding data-MC checks, etc. In this phase space, we will use a binned summary observable like in any traditional analysis.

importlib.reload(sys.modules['utils'])
from utils import preselection_using_score


dataset_incl_nominal['presel_score'] = presel_score

channel_selections = {'CR': {'observable': 'presel_score', 
                             'lower_presel': -999, 
                             'upper_presel': preselection_cuts.get('lower'), 
                             'num_bins': 4},
                      
                      'SR_binned': {'observable': 'presel_score', 
                                    'lower_presel': preselection_cuts.get('upper'), 
                                    'upper_presel': -999,
                                    'num_bins': 1},
                      
                      'SR': {'observable': None, 
                             'upper_presel': preselection_cuts.get('upper'), 
                             'lower_presel': preselection_cuts.get('lower')}}

dataset_channels = preselection_using_score(dataset_incl_nominal, channel_selections)

del dataset_incl_nominal